### Travaux Pratiques sur le chiffrement DES
___
#### Auteurs: B3LIOTT et Altrusc
#### Date: 16/02/2023
#### Matière: Cryptographie
___

In [12]:
import random 
from numpy import array

La fonction $StringToBinary$ ne fait pas partie du DES, elle permetera uniquement de visualiser le chiffrement. Elle prend en paramètre une chaine de caractère isible,  et la retourne sous la forme de mots p tels que $p = p_1p_2\cdots p_{64}  \in\{0,1\}^{64}$

In [277]:
def StringToBinary(string):
    p = ''.join(format(ord(i), '08b') for i in string)
    listP = []
    interP = ""
    for i in range(1, len(p)+1):
        interP += p[i-1]
        if i%64 == 0:
            listP.append(interP)
            interP = ""
    
    if interP != "":
        listP.append(interP)
    
    # Ajout d'espaces si la taille du dernier bloc est inférieure à 64
    listP[-1] += "00100000"*(8-len(listP[-1])//8)

    return listP

In [270]:
#Test
Message = "Bonjournnnndnnndn"
print(StringToBinary(Message))

8
['0100001001101111011011100110101001101111011101010111001001101110', '0110111001101110011011100110010001101110011011100110111001100100', '0110111000100000001000000010000000100000001000000010000000100000']


La fonction $clearRandomMessage$ retourne un message aléatoire de 64 bits.

In [16]:
def clearRandomMessage():
    msg = ""
    for i in range(0, 64):
        msg += str(random.randint(0, 1))
    return msg

In [17]:
#Test
print(clearRandomMessage())

0010100000101100001001001110011111111011101111010011000000010111


In [18]:
def afficheOctets(p):
    s = ""
    for i in range(len(p)):
        if i%8 == 0:
            s += "\n" + "Octet " + str(i//8) + ": "

        s += str(p[i])

    print(s)    


In [19]:
#Test
afficheOctets(StringToBinary(Message))
print()
afficheOctets(clearRandomMessage())


Octet 0: 01000010
Octet 1: 01101111
Octet 2: 01101110
Octet 3: 01101010
Octet 4: 01101111
Octet 5: 01110101
Octet 6: 01110010
Octet 7: 00100000


Octet 0: 01010001
Octet 1: 01000111
Octet 2: 00001110
Octet 3: 11011100
Octet 4: 11011001
Octet 5: 00010100
Octet 6: 00100010
Octet 7: 00110000


La fonction $genOctetKey$ génère un octet aléatoire dont la somme des bits est impaire (vous comprendrez en lisant le paragraphe suivant sur l'ensemble K des clés).

In [20]:
def genOctetKey():
    k = ""
    randomInd = 0
    randomImpaire = 1
    indList = [0,1,2,3,4,5,6,7]
    bitList = [0]*8
    randomImpaire = 2*random.randint(0,3) + 1 #nombre impair aléatoire qui définira la somme des bits de l'octet
        
    for i in range(randomImpaire):
        randomInd = random.randint(0,len(indList)-1) #indice aléatoire parmis ceux qui restent pour placer le bit dans l'octet
        bitList[indList[randomInd]] = 1
        indList.remove(indList[randomInd])

    # formatage sous forme d'une chaine de caractères
    for bit in bitList:
        k += str(bit)
        
    return k

In [25]:
#Test
afficheOctets(genOctetKey())


Octet 0: 00010000


La fonction $genKey$ génère une clé de session k (DES de 64 bits) aléatoirement.  

La clé k est telle que: 
\begin{align*}
k \in K = \{(b_1b_2\cdots b_{64}) \in \{0,1\}^{64} \quad \text{|} \quad \forall j=0\dots7, \sum_{i=1}^{8}b_{8j+i} \equiv 1 \pmod 2 \}
\end{align*}

In [26]:
def genKey():
    k = ""

    # On génère 8 octets pour la clé de 64 bits
    for i in range(8):
        k += genOctetKey()
    
    return k

In [27]:
#Test
afficheOctets(genKey())


Octet 0: 11111110
Octet 1: 01011101
Octet 2: 01111001
Octet 3: 10100100
Octet 4: 01111111
Octet 5: 00100000
Octet 6: 10011110
Octet 7: 00100011


In [29]:
#phi1:
PC1g = array([57,49,41,33,25,17,9,
1,58,50,42,34,26,18,
10,2,59,51,43,35,27,
19,11,3,60,52,44,36])

PC1d = array([63,55,47,39,31,23,15,
7,62,54,46,38,30,22,
14,6,61,53,45,37,29,
21,13,5,28,20,12,4])

#phi2:
PC2 = array([14,17,11,24,1,5,
3,28,15,6,21,10,
23,19,12,4,26,8,
16,7,27,20,13,2,
41,52,31,37,47,55,
30,40,51,45,33,48,
44,49,39,56,34,53,
46,42,50,36,29,32])

La fonction $phi1$ prend en paramètre une clé k et retourne le couple $(C, D)$ où $C$ et $D$ sont les permutations de certains éléments de $k$ selon la table de permutation $PC_{1g}$ (pour C) et $PC_{2d}$ (pour D).

In [30]:
def phi1(k):
    C = ""
    D = ""
    for data in PC1g:
        C += k[data-1]
        
    for data in PC1d:
        D += k[data-1]

    return (C, D)

La fonction $phi2$ prend en paramètre un couple $(C, D)$ et retourne la clé $k$ de longueur 48 (56 avec 8 mots oubliés et les autres permutés) obtenue par la concaténation de $C$ et $D$ selon la table de permutation $PC_{2}$.

In [31]:
def phi2(couple):
    mot = "" 
    
    # Concatenation:
    for i in range(28):
        mot += couple[0][i]
        
    for j in range(28):    
        mot += couple[1][j]
    
    # Permutation:
    k = ""
    for data in PC2:
        k += mot[data-1]
    
    return k


In [33]:
#Test
sessionKey = genKey()

print("Phi1: ",phi1(sessionKey), "\n")
print("Phi2: ",phi2(phi1(sessionKey)))

print("Taille Phi2: ",len(phi2(phi1(sessionKey))), "\n")

Phi1:  ('1110111110100111011010100110', '0010011101100101001110011001') 

Phi2:  101011101110110011111001011010000001111111010100
Taille Phi2:  48 



Les S-boites sont les boites de substitution suivantes:

In [34]:
S1 = array([
[14,4,13,1,2,15,11,8,3,10,6,12,5,9,0,7],
[0,15,7,4,14,2,13,1,10,6,12,11,9,5,3,8],
[4,1,14,8,13,6,2,11,15,12,9,7,3,10,5,0],
[15,12,8,2,4,9,1,7,5,11,3,14,10,0,6,13]])

S2 = array([
[15,1,8,14,6,11,3,4,9,7,2,13,12,0,5,10],
[3,13,4,7,15,2,8,14,12,0,1,10,6,9,11,5],
[0,14,7,11,10,4,13,1,5,8,12,6,9,3,2,15],
[13,8,10,1,3,15,4,2,11,6,7,12,0,5,14,9]])

S3 = array([
[10,0,9,14,6,3,15,5,1,13,12,7,11,4,2,8],
[13,7,0,9,3,4,6,10,2,8,5,14,12,11,15,1],
[13,6,4,9,8,15,3,0,11,1,2,12,5,10,14,7],
[1,10,13,0,6,9,8,7,4,15,14,3,11,5,2,12]])

S4 = array([
[7,13,14,3,0,6,9,10,1,2,8,5,11,12,4,15],
[13,8,11,5,6,15,0,3,4,7,2,12,1,10,14,9],
[10,6,9,0,12,11,7,13,15,1,3,14,5,2,8,4],
[3,15,0,6,10,1,13,8,9,4,5,11,12,7,2,14]])

S5 = array([
[2,12,4,1,7,10,11,6,8,5,3,15,13,0,14,9],
[14,11,2,12,4,7,13,1,5,0,15,10,3,9,8,6],
[4,2,1,11,10,13,7,8,15,9,12,5,6,3,0,14],
[11,8,12,7,1,14,2,13,6,15,0,9,10,4,5,3]])

S6 = array([
[12,1,10,15,9,2,6,8,0,13,3,4,14,7,5,11],
[10,15,4,2,7,12,9,5,6,1,13,14,0,11,3,8],
[9,14,15,5,2,8,12,3,7,0,4,10,1,13,11,6],
[4,3,2,12,9,5,15,10,11,14,1,7,6,0,8,13]])

S7 = array([
[4,11,2,14,15,0,8,13,3,12,9,7,5,10,6,1],
[13,0,11,7,4,9,1,10,14,3,5,12,2,15,8,6],
[1,4,11,13,12,3,7,14,10,15,6,8,0,5,9,2],
[6,11,13,8,1,4,10,7,9,5,0,15,14,2,3,12]])

S8 = array([
[13,2,8,4,6,15,11,1,10,9,3,14,5,0,12,7],
[1,15,13,8,10,3,7,4,12,5,6,11,0,14,9,2],
[7,11,4,1,9,12,14,2,0,6,10,13,15,3,5,8],
[2,1,14,7,4,10,8,13,15,12,9,0,3,5,6,11]])

L'Expension E est représentée par la table suivante:

In [35]:
E = array([32 ,1 ,2 ,3 ,4 ,5,
4 ,5 ,6 ,7 ,8 ,9,
8 ,9 ,10 ,11 ,12 ,13,
12 ,13 ,14 ,15 ,16 ,17,
16 ,17 ,18 ,19 ,20 ,21,
20 ,21 ,22 ,23 ,24 ,25,
24 ,25 ,26 ,27 ,28 ,29,
28 ,29 ,30 ,31 ,32 ,1 ])

La permutation de sortie P est représentée par la table suivante:

In [36]:
P  = array([16,7,20,21,
29,12,28,17,
1,15,23,26,
5,18,31,10,
2,8,24,14,
32,27,3,9,
19,13,30,6,
22,11,4,25])

La fonction $IntToBinary$ retourne un entier $n \in\{0,15\}$ sous la forme d'une chaine de caractère binaire de taille $4$.

In [37]:
def IntToBinary(n):
    word = bin(n).replace("0b", "")
    return word.zfill(4)

In [38]:
#test
print (IntToBinary(1))
print(IntToBinary(7))
print(IntToBinary(12))
print(IntToBinary(15))

0001
0111
1100
1111


On cherche à déterminer le couple $(L_0, R_0)$ (pour left et right) à partir de $p$ et de la clé $k$. On utilise la permutation initiale $\sigma$ pour permuter les bits de $p$.

In [84]:
sigma = array(
    [58,50,42,34,26,18,10,2,60,52,44,36,28,20,12,4,62,54,46,38,30,22,14,6,64,56,48,40,32,24,16,8,
     57,49,41,33,25,17,9,1,59,51,43,35,27,19,11,3,61,53,45,37,29,21,13,5,63,55,47,39,31,23,15,7], dtype = int)

sigma_moins_un = [40, 8, 48, 16, 56, 24, 64, 32, 39, 7, 47, 15, 55, 23, 63, 31, 38, 6, 46, 14,
                  54, 22, 62, 30, 37, 5, 45, 13, 53, 21, 61, 29, 36, 4, 44, 12, 52, 20, 60, 28,
                  35, 3, 43, 11, 51, 19, 59, 27, 34, 2, 42, 10, 50, 18, 58, 26, 33, 1, 41, 9, 49, 17, 57, 25]

In [112]:
def sigma0(word):
    result = ""
    
    # Le bit d'indice i est remplacé par le bit d'indice sigma-1(i):
    for i in range(64):
        result += word[sigma_moins_un[i]-1]
    
    return (result[:32], result[32:])

In [137]:
def sigma0_moins_un(couple):
    word = couple[0] + couple[1]
    result = ""

    # Le bit d'indice i est remplacé par le bit d'indice sigma(i):
    for i in range(64):
        result += word[sigma[i]-1]

    return result

In [41]:
#Test
word = clearRandomMessage()
print("Message :", word)
L, R = sigma0(word)
print("L :", L, "et R :", R)

Message : 0100100101101000100101001101011111110101111111111110011111000000
L : 1110100100101001101011010111000 et R : 10100101101110001111101110101111


La fonction B calcule E(R) XOR K à partir d'un mot R de longueur 32 et d'une clé K de longueur 48

In [42]:
def B(R, K):
    #Expension de R
    R = [R[i-1] for i in E]
    #XOR de R et K
    R = [int(R[i]) ^ int(K[i]) for i in range(len(R))]
    res = ""
    for bit in R:
        res += str(bit)
    return res

In [65]:
#Test
print("Le mot B obtenu: ",B(R, sessionKey))

Le mot B obtenu:  000011110101111000110110110001100111010110100001


La fonction $decoupe$ permet de decouper le mot B en B1,B2...B8

In [44]:
def decoupe(B):
    listBi = []
    Bi = ""
    for i in range(48):
        if i%6 == 0 and i != 0:
            listBi.append(Bi)
            Bi = ""
        
        Bi += B[i]
    
    listBi.append(Bi)
    
    return listBi

In [46]:
#Test
print(decoupe("000000000000000000001111111111111111111100011101"))

['000000', '000000', '000000', '001111', '111111', '111111', '111100', '011101']


La fonction $C$ prend en paramètre la liste des 8 $B_i$, chacun de longueur 6, calcule les $C_i = S_i(B_i)$ puis concatene les $C_i$ pour obtenir $C$.

In [61]:
def C(B):
    mot = ""
    listSi = [S1, S2, S3, S4, S5, S6, S7, S8]

    for i in range(8):
        indLigne = int(B[i][0])*2 + int(B[i][5])
        indColonne = int(B[i][1])*8 + int(B[i][2])*4 + int(B[i][3])*2 + int(B[i][4])
        mot += IntToBinary(listSi[i][indLigne, indColonne])
    
    return mot

In [63]:
#Test 
print("Concaténation des Ci: "+ C(decoupe("000000000000000000001111111111111111111100011101")))


Concaténation des Ci: 11101111101000110011110110011001


In [64]:
def fki(R, ki):
    B_word = B(R, ki)
    listB = decoupe(B_word)
    C_word = C(listB)
    
    # permutation P(C):
    C_word = [C_word[i-1] for i in P]
    res = ""
    for bit in C_word:
        res += str(bit)
        
    return res 

In [69]:
#Test
print("Le mot fki obtenu: ",fki(R, sessionKey))

Le mot fki obtenu:  01010110110101111101011011011010


La fonction tour permet de générer un nouveau couple (Li, Ri) à partir de (Li-1, Ri-1).
Elle prent en paramètre la clé de tour ki et le couple (Li-1, Ri-1).

In [125]:
def tour(couple, ki):
    R = couple[1]
    L = couple[0]
    newL = R
    R_prime = fki(R, ki)
    newRbits = [int(L[i]) ^ int(R_prime[i]) for i in range(len(R_prime))]
    newR = ""
    for bit in newRbits:
        newR += str(bit)

    return (newL, newR)

In [72]:
def permutaion_circlulaire(word, n):
    return word[n:] + word[:n]


La fonction $genAllKi$ est une fonction qui génère les 16 clés de tour pour pouvoir les utiliser par la suite, qui fait appel à phi1 et phi2 et qui effectue une permutation pour chaque clé sur (C, D) en fonction de $v_i$:

Pour $i = 1,\cdots,16$
\begin{align*}
v_i =
\begin{cases}
1 & \text{si } i \in \{1,2,9,16\} \\
2 & \text{sinon}
\end{cases}
\end{align*}

In [80]:
def genAllKi(K):
    testList = [1, 2, 9, 16]
    kiList = []
    couple = phi1(K)

    for i in range(16):      
        if i in testList:
            perm = 1
        else:
            perm = 2

        couple = (permutaion_circlulaire(couple[0], perm), permutaion_circlulaire(couple[1], perm))
        kiList.append(phi2(couple))
        
    return kiList

In [81]:
#Test 
listKi = genAllKi(genKey())
print("Voici la liste des 16 clés de tour: \n")
print(listKi)

Voici la liste des 16 clés de tour: 

['100110110010110011000001001010001000001000000000', '000010010010100000011111000000100000000000101000', '010110010110011010001100000100000100010000000010', '010100001001010110001100000011000000000000000000', '010100001000100001100111100000000110000001000000', '101000011110100000100110001000001000001000000000', '101000000010011110100110100100000000010000000010', '111100000001011000110001000011000000001000000000', '110001011001001001110000000100000110000001000000', '101000001110000101111010010000000000100000000101', '101001000100011100010011000000100000000010011000', '011001110001101100010001000000010001000100000001', '000011111011000011010001000000100000000000100000', '000111110100010011011010010000000000100100000100', '011111100100000110001000000000000000000010011000', '000110101000100100001101010000010001000000000001']


La fonction $DES$ prend en paramètre un mot en clair binaire et retourne le mot correspondant chiffré par la methode DES.

In [129]:
def DES(p):
    K = genKey()
    couple = sigma0(p)
    kiList = genAllKi(K)

    # On effectue les 16 tours:
    for i in range(16):
        couple = tour(couple, listKi[i])
    
    # Permutation finale:
    return sigma0_moins_un(couple)

In [181]:
#Test
p = clearRandomMessage()
print("Message en clair:        " + p)
print("Message chiffré par DES: " + DES(p))

Message en clair:        0011100000101011100011101101101001011100000010100111110101100001
Message chiffré par DES: 0010100001011111101110011101110010101011000101100100000101110010


La fonction $chiffrementChaines$ ne fait pas partie du DES, elle permet uniquement de nous aider à visualiser le chiffremement. Elle prend en paramètre une chaine de caractères lisible par l'Homme (i.e non binaire) et retourne la chaine associée chiffrée par DES.

In [152]:
def BinaryToString(binary):
    res = ""
    for i in range(0, len(binary), 8):
        res += chr(int(binary[i:i+8], 2))
    return res

In [275]:
def chiffrementChaines(word):
    listP = StringToBinary(word)
    result = ""

    for p in listP:
        result += DES(p)
    
    return BinaryToString(result)


In [278]:
#Test
print("Chiffrement de la chaîne 'Bonjour tout le monde !' :")
print(chiffrementChaines("Bonjour tout le monde !"))


Chiffrement de la chaîne 'Bonjour tout le monde !' :
&ì-HEð·¶åÚP¡.(ÑvÒ×
